In [1]:

import os
import dotenv
from datetime import datetime
import pandas as pd
import numpy as np
import time

dotenv.load_dotenv()

True

#### These functions were used to create the file_name and the file_path in google_travel_time.ipynb

In [15]:
hours = [
    datetime(2024, 2, 26, 7, 30, 0),
    datetime(2024, 2, 26, 12, 00, 0),
    datetime(2024, 2, 26, 16, 00, 0),
    datetime(2024, 2, 26, 20, 15, 0)

]

def get_variable_name(curr_hour: datetime, mode: str) -> str:

    hour = f"{curr_hour.hour:02d}"
    minute = f"{curr_hour.minute:02d}"

    column_name_for_time_at_this_time = f"T{hour}{minute}_{mode}_(In Minutes)"
    output_file_name = f"T{hour}{minute}_{mode}.parquet"

    return column_name_for_time_at_this_time, output_file_name


In [17]:
hours = [
    datetime(2024, 2, 26, 7, 30, 0),
    datetime(2024, 2, 26, 12, 00, 0),
    datetime(2024, 2, 26, 16, 00, 0),
    datetime(2024, 2, 26, 20, 15, 0)
]

modes = ["transit", "bicycling"]

time_compute = hours[0]

#### Get the common columns from one of the parquet files

- Also extract the travel distance from one of the `transit` file and `bicycle` file

In [139]:
## Extract these column from one of the bicycle dataframes

def extract_common_columns(mode: str, hours) -> pd.DataFrame:
    col_name, output_file_name = get_variable_name(hours[0], mode)
    cols_common_bike = ['Destination_Coordinates', 'Origin_Coordinates', 'Distance (In Meters)']
    df = pd.read_parquet(os.path.join(os.environ['OUTPUT_PATH'], "google_hourly_data", output_file_name))

    return df[cols_common_bike].copy()

Loop all `mode` files and combine them into one file

In [140]:
def combine_all_hours(mode:str, hours: list) -> pd.DataFrame:
    df_combined = extract_common_columns(mode, hours)

    for i in hours:
        col_name, output_file_name = get_variable_name(i, mode)
        try:
            df = pd.read_parquet(os.path.join(os.environ['OUTPUT_PATH'], "google_hourly_data", output_file_name))
            df_curr = df[['Destination_Coordinates', 'Origin_Coordinates']+ [col_name]].copy()
            df_combined = df_combined.merge(df_curr, on=['Destination_Coordinates', 'Origin_Coordinates'], how='left')
        except:
            print(f"File {output_file_name} does not exist")
    
    return df_combined.drop_duplicates()


In [141]:
df_transit = combine_all_hours("transit", hours)
df_transit.rename(columns={"Distance (In Meters)": f"Distance (In Meters) (transit)"}, inplace=True)
df_bicycling = combine_all_hours("bicycling", hours)
df_bicycling.rename(columns={"Distance (In Meters)": f"Distance (In Meters) (bicycling)"}, inplace=True)

File T2015_transit.parquet does not exist


In [142]:
df_transit.shape, df_bicycling.shape

((13225, 6), (13225, 7))

In [143]:
df_combined_transit_bicycling = df_transit.merge(df_bicycling, on=['Destination_Coordinates', 'Origin_Coordinates'], how='right')

In [145]:
df_combined_transit_bicycling.to_parquet(os.path.join(os.environ['OUTPUT_PATH'], "combined_google_transit_bicycling.parquet"))